Resnet_50

Sarees attr_4 using Resnet_50 for extracting features and predictions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    'Sarees': ['attr_4'],
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator():
    return ImageDataGenerator(rescale=1./255)

# Define feature extraction function using ResNet50
def extract_features(image_files, image_dir, model, target_size=(224, 224), batch_size=32):
    data_gen = create_data_generator()
    generator = data_gen.flow_from_dataframe(
        dataframe=image_files, directory=image_dir, x_col='filename', y_col=None,
        target_size=target_size, batch_size=batch_size, class_mode=None, shuffle=False
    )
    features = model.predict(generator, verbose=1)
    return features

# Load pre-trained ResNet50 without the fully connected layers, for feature extraction
feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        num_classes = df_attr[attr].nunique()
        binary = (num_classes == 2)
        
        # Use label encoding for all attributes
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_accuracy_across_folds = 0
        best_model_path = f'best_knn_model_{category}_{attr}.pkl'
        
        # Loop for each fold
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Extract features for train and validation sets
            train_features = extract_features(train_fold[['filename']], train_image_dir, feature_extractor)
            val_features = extract_features(val_fold[['filename']], train_image_dir, feature_extractor)
            
            # Train KNN model
            knn = KNeighborsClassifier(n_neighbors=7)  # You can adjust the number of neighbors based on experiments
            knn.fit(train_features, train_fold['label'])
            
            # Validate the model
            val_preds = knn.predict(val_features)
            val_accuracy = accuracy_score(val_fold['label'], val_preds)
            print(f"Validation accuracy for fold {fold+1}: {val_accuracy}")

            # Check if this fold's accuracy is the best so far
            if val_accuracy > best_accuracy_across_folds:
                best_accuracy_across_folds = val_accuracy
                # Save the model here if you want, e.g., using joblib or pickle
                # joblib.dump(knn, best_model_path)
                print(f"New best model for {category} - {attr} with accuracy: {best_accuracy_across_folds}")

        print(f"Best model across all folds for {category} - {attr} with accuracy: {best_accuracy_across_folds}")

# ---- Prediction on Test Set ----
# Example prediction flow
df_test['filename'] = df_test['id'].astype(str).apply(lambda x: x.zfill(6) + '.jpg')
for category, attributes in categories_attributes.items():
    for attr in attributes:
        print(f"Predicting on test set for category: {category}, attribute: {attr}")
        
        # Load the best model (if saved)
        # knn = joblib.load(best_model_path)
        
        # Extract features for the test set
        test_features = extract_features(df_test[['filename']], test_image_dir, feature_extractor)
        
        # Make predictions
        test_preds = knn.predict(test_features)
        
        # Convert labels back to original categories
        test_preds_decoded = label_encoders[category][attr].inverse_transform(test_preds)
        df_test[f'predicted_{attr}'] = test_preds_decoded

# Save test predictions
df_test[['id'] + [f'predicted_{attr}' for attr in categories_attributes['Sarees']]].to_csv('test_predictions.csv', index=False)
print("Test predictions saved to 'test_predictions.csv'")


Sarees attr_5 using Resnet_50 for extracting features and predictions

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    #'Men Tshirts': ['attr_1','attr_2', 'attr_3', 'attr_4','attr_5'],
    'Sarees': ['attr_5'],
    #'Kurtis': [ 'attr_2'],
    #'Women Tshirts': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8'],
    #'Women Tops and Tunics': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator(train=True):
    if train:
        return ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                  shear_range=0.2, zoom_range=0.4, horizontal_flip=True, fill_mode='nearest')
    else:
        return ImageDataGenerator(rescale=1./255)

# Define model-building function
def build_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Sequential([
        base_model,
        Flatten(),
        Dropout(0.5),
    ])
    #for layer in base_model.layers[:]:
    #    layer.trainable = True
    #for layer in base_model.layers[:-20]:
     #   layer.trainable = False   
    
    # Add L2 regularization with a lambda value (you can adjust the value)
    l2_lambda = 0.001  # Example value; you can adjust it based on your experiments
    
    if num_classes == 2:  # Binary classification
        model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    else:  # Multi-class classification with sparse labels
       # model.add(Dense(256, activation='relu'))
       # model.add(Dropout(0.4))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model
# Training process for each category and attribute
# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        num_classes = df_attr[attr].nunique()
        binary = (num_classes == 2)
        
        # Use label encoding for all attributes
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le
        if binary:
            df_attr['label'] = df_attr['label'].astype(str)

        # Calculate class weights
       # class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(df_attr['label']), y=df_attr['label'])
        #class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_accuracy_across_folds = 0
        best_model_path = f'best_model_{category}_{attr}.h5'
        
        # Loop for each fold
        best_fold_accuracy = 0
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Create data generators
            train_generator = create_data_generator(train=True).flow_from_dataframe(
                dataframe=train_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw')

            val_generator = create_data_generator(train=False).flow_from_dataframe(
                dataframe=val_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw', shuffle=False)

            # Build the model
            model = build_model(num_classes=num_classes)

            # Set up variables to track the best model in this fold
            
            best_fold_model_path = f'best_model_{category}_{attr}_fold{fold+1}.h5'
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1)
            

            # Train for multiple epochs and save the model for the best epoch in this fold
            for epoch in range(10):
                print(f"Epoch {epoch+1}/{10}")
                history = model.fit(
                    train_generator, validation_data=val_generator, epochs=1, verbose=1,callbacks=[early_stopping, reduce_lr])

                # Get the validation accuracy of the current epoch
                val_accuracy = history.history['val_accuracy'][0]

                # If the current epoch's accuracy is the best, save the model for this epoch
                if val_accuracy > best_fold_accuracy:
                    best_fold_accuracy = val_accuracy
                    model.save(best_fold_model_path)
                    print(f"New best model saved for fold {fold+1} at epoch {epoch+1} with accuracy: {val_accuracy}")

            # After the fold, check if this fold's best model is better than previous folds
            if best_fold_accuracy > best_accuracy_across_folds:
                best_accuracy_across_folds = best_fold_accuracy
                best_model_path = best_fold_model_path
                print(f"Best model across all folds so far for {category} - {attr} saved with accuracy: {best_accuracy_across_folds}")

        print(f"Overall best model for {category} - {attr} saved at: {best_model_path} with accuracy: {best_accuracy_across_folds}")
# ---- Prediction on Test Set ----

Sarees attr_6 using Resnet_50 for extracting features and predictions

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    #'Men Tshirts': ['attr_1','attr_2', 'attr_3', 'attr_4','attr_5'],
    'Sarees': ['attr_6'],
    #'Kurtis': [ 'attr_2'],
    #'Women Tshirts': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8'],
    #'Women Tops and Tunics': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator(train=True):
    if train:
        return ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                  shear_range=0.2, zoom_range=0.4, horizontal_flip=True, fill_mode='nearest')
    else:
        return ImageDataGenerator(rescale=1./255)

# Define model-building function
def build_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Sequential([
        base_model,
        Flatten(),
        Dropout(0.5),
    ])
    #for layer in base_model.layers[:]:
    #    layer.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False   
    
    # Add L2 regularization with a lambda value (you can adjust the value)
    l2_lambda = 0.001  # Example value; you can adjust it based on your experiments
    
    if num_classes == 2:  # Binary classification
        model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
    else:  # Multi-class classification with sparse labels
       # model.add(Dense(256, activation='relu'))
       # model.add(Dropout(0.4))
       # model.add(Dense(128, activation='relu'))
       # model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model
# Training process for each category and attribute
# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        num_classes = df_attr[attr].nunique()
        binary = (num_classes == 2)
        
        # Use label encoding for all attributes
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le
        if binary:
            df_attr['label'] = df_attr['label'].astype(str)

        # Calculate class weights
       # class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(df_attr['label']), y=df_attr['label'])
        #class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_accuracy_across_folds = 0
        best_model_path = f'best_model_{category}_{attr}.h5'
        
        # Loop for each fold
        best_fold_accuracy = 0
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Create data generators
            train_generator = create_data_generator(train=True).flow_from_dataframe(
                dataframe=train_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw')

            val_generator = create_data_generator(train=False).flow_from_dataframe(
                dataframe=val_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw', shuffle=False)

            # Build the model
            model = build_model(num_classes=num_classes)

            # Set up variables to track the best model in this fold
            
            best_fold_model_path = f'best_model_{category}_{attr}_fold{fold+1}.h5'
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1)
            

            # Train for multiple epochs and save the model for the best epoch in this fold
            for epoch in range(10):
                print(f"Epoch {epoch+1}/{10}")
                history = model.fit(
                    train_generator, validation_data=val_generator, epochs=1, verbose=1,callbacks=[early_stopping, reduce_lr])

                # Get the validation accuracy of the current epoch
                val_accuracy = history.history['val_accuracy'][0]

                # If the current epoch's accuracy is the best, save the model for this epoch
                if val_accuracy > best_fold_accuracy:
                    best_fold_accuracy = val_accuracy
                    model.save(best_fold_model_path)
                    print(f"New best model saved for fold {fold+1} at epoch {epoch+1} with accuracy: {val_accuracy}")

            # After the fold, check if this fold's best model is better than previous folds
            if best_fold_accuracy > best_accuracy_across_folds:
                best_accuracy_across_folds = best_fold_accuracy
                best_model_path = best_fold_model_path
                print(f"Best model across all folds so far for {category} - {attr} saved with accuracy: {best_accuracy_across_folds}")

        print(f"Overall best model for {category} - {attr} saved at: {best_model_path} with accuracy: {best_accuracy_across_folds}")
# ---- Prediction on Test Set ----

Sarees attr_7 using Resnet_50 for extracting features and predictions

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    #'Men Tshirts': ['attr_1','attr_2', 'attr_3', 'attr_4','attr_5'],
    'Sarees': ['attr_7'],
    #'Kurtis': [ 'attr_2'],
    #'Women Tshirts': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8'],
    #'Women Tops and Tunics': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator(train=True):
    if train:
        return ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                  shear_range=0.2, zoom_range=0.4, horizontal_flip=True, fill_mode='nearest')
    else:
        return ImageDataGenerator(rescale=1./255)

# Define model-building function
def build_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Sequential([
        base_model,
        Flatten(),
        Dropout(0.5),
    ])
    #for layer in base_model.layers[:]:
     #   layer.trainable = True
    #for layer in base_model.layers[:-20]:
    #    layer.trainable = False   
    
    # Add L2 regularization with a lambda value (you can adjust the value)
    l2_lambda = 0.001  # Example value; you can adjust it based on your experiments
    
    if num_classes == 2:  # Binary classification
      #  model.add(Dense(256, activation='relu'))
       # model.add(Dropout(0.4))
        #model.add(Dense(128, activation='relu'))
        #model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    else:  # Multi-class classification with sparse labels
        #model.add(Dense(256, activation='relu'))
        #model.add(Dropout(0.4))
        #model.add(Dense(128, activation='relu'))
        #model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model
# Training process for each category and attribute
# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        num_classes = df_attr[attr].nunique()
        binary = (num_classes == 2)
        
        # Use label encoding for all attributes
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le
        if binary:
            df_attr['label'] = df_attr['label'].astype(str)

        # Calculate class weights
       # class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(df_attr['label']), y=df_attr['label'])
        #class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_accuracy_across_folds = 0
        best_model_path = f'best_model_{category}_{attr}.h5'
        
        # Loop for each fold
        best_fold_accuracy = 0
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Create data generators
            train_generator = create_data_generator(train=True).flow_from_dataframe(
                dataframe=train_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw')

            val_generator = create_data_generator(train=False).flow_from_dataframe(
                dataframe=val_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw', shuffle=False)

            # Build the model
            model = build_model(num_classes=num_classes)

            # Set up variables to track the best model in this fold
            
            best_fold_model_path = f'best_model_{category}_{attr}_fold{fold+1}.h5'
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1)
            

            # Train for multiple epochs and save the model for the best epoch in this fold
            for epoch in range(10):
                print(f"Epoch {epoch+1}/{10}")
                history = model.fit(
                    train_generator, validation_data=val_generator, epochs=1, verbose=1,callbacks=[early_stopping, reduce_lr])

                # Get the validation accuracy of the current epoch
                val_accuracy = history.history['val_accuracy'][0]

                # If the current epoch's accuracy is the best, save the model for this epoch
                if val_accuracy > best_fold_accuracy:
                    best_fold_accuracy = val_accuracy
                    model.save(best_fold_model_path)
                    print(f"New best model saved for fold {fold+1} at epoch {epoch+1} with accuracy: {val_accuracy}")

            # After the fold, check if this fold's best model is better than previous folds
            if best_fold_accuracy > best_accuracy_across_folds:
                best_accuracy_across_folds = best_fold_accuracy
                best_model_path = best_fold_model_path
                print(f"Best model across all folds so far for {category} - {attr} saved with accuracy: {best_accuracy_across_folds}")

        print(f"Overall best model for {category} - {attr} saved at: {best_model_path} with accuracy: {best_accuracy_across_folds}")
# ---- Prediction on Test Set ----

Sarees attr_8 using Resnet_50 for extracting features and predictions

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    # 'Men Tshirts': ['attr_1','attr_2', 'attr_3', 'attr_4','attr_5'],
    'Sarees': ['attr_8'],
    # 'Kurtis': [ 'attr_2'],
    # 'Women Tshirts': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8'],
    # 'Women Tops and Tunics': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator(train=True):
    if train:
        return ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                  shear_range=0.2, zoom_range=0.4, horizontal_flip=True, fill_mode='nearest')
    else:
        return ImageDataGenerator(rescale=1./255)

# Define model-building function
def build_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Sequential([
        base_model,
        Flatten(),
        Dropout(0.5),
    ])
    for layer in base_model.layers[:]:
        layer.trainable = True
    #for layer in base_model.layers[:-20]:
    #    layer.trainable = False   
    
    # Add L2 regularization with a lambda value (you can adjust the value)
    l2_lambda = 0.01  # Example value; you can adjust it based on your experiments
    
    if num_classes == 2:  # Binary classification
       # model.add(Dense(256, activation='relu'))
        #model.add(Dropout(0.4))
        #model.add(Dense(128, activation='relu'))
        #model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
    else:  # Multi-class classification with sparse labels
        #model.add(Dense(256, activation='relu'))
        #model.add(Dropout(0.4))
        #model.add(Dense(128, activation='relu'))
        #model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model
# Training process for each category and attribute
# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        num_classes = df_attr[attr].nunique()
        binary = (num_classes == 2)
        
        # Use label encoding for all attributes
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le
        if binary:
            df_attr['label'] = df_attr['label'].astype(str)

        # Calculate class weights
       # class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(df_attr['label']), y=df_attr['label'])
        #class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_accuracy_across_folds = 0
        best_model_path = f'best_model_{category}_{attr}.h5'
        
        # Loop for each fold
        best_fold_accuracy = 0
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Create data generators
            train_generator = create_data_generator(train=True).flow_from_dataframe(
                dataframe=train_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw')

            val_generator = create_data_generator(train=False).flow_from_dataframe(
                dataframe=val_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw', shuffle=False)

            # Build the model
            model = build_model(num_classes=num_classes)

            # Set up variables to track the best model in this fold
            
            best_fold_model_path = f'best_model_{category}_{attr}_fold{fold+1}.h5'
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1)
            

            # Train for multiple epochs and save the model for the best epoch in this fold
            for epoch in range(10):
                print(f"Epoch {epoch+1}/{10}")
                history = model.fit(
                    train_generator, validation_data=val_generator, epochs=1, verbose=1,callbacks=[early_stopping, reduce_lr])

                # Get the validation accuracy of the current epoch
                val_accuracy = history.history['val_accuracy'][0]

                # If the current epoch's accuracy is the best, save the model for this epoch
                if val_accuracy > best_fold_accuracy:
                    best_fold_accuracy = val_accuracy
                    model.save(best_fold_model_path)
                    print(f"New best model saved for fold {fold+1} at epoch {epoch+1} with accuracy: {val_accuracy}")

            # After the fold, check if this fold's best model is better than previous folds
            if best_fold_accuracy > best_accuracy_across_folds:
                best_accuracy_across_folds = best_fold_accuracy
                best_model_path = best_fold_model_path
                print(f"Best model across all folds so far for {category} - {attr} saved with accuracy: {best_accuracy_across_folds}")

        print(f"Overall best model for {category} - {attr} saved at: {best_model_path} with accuracy: {best_accuracy_across_folds}")
# ---- Prediction on Test Set ----

Sarees attr_9 using Resnet_50 for extracting features and predictions

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    #'Men Tshirts': ['attr_1','attr_2', 'attr_3', 'attr_4','attr_5'],
    'Sarees': ['attr_9'],
    #'Kurtis': [ 'attr_2'],
    #'Women Tshirts': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8'],
    #'Women Tops and Tunics': ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator(train=True):
    if train:
        return ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                  shear_range=0.2, zoom_range=0.4, horizontal_flip=True, fill_mode='nearest')
    else:
        return ImageDataGenerator(rescale=1./255)

# Define model-building function
def build_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Sequential([
        base_model,
        Flatten(),
        Dropout(0.5),
    ])
    for layer in base_model.layers[:]:
        layer.trainable = True
    #for layer in base_model.layers[:-20]:
    #    layer.trainable = False   
    
    # Add L2 regularization with a lambda value (you can adjust the value)
    l2_lambda = 0.01  # Example value; you can adjust it based on your experiments
    
    if num_classes == 2:  # Binary classification
       # model.add(Dense(256, activation='relu'))
        #model.add(Dropout(0.4))
        #model.add(Dense(128, activation='relu'))
        #model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
    else:  # Multi-class classification with sparse labels
        #model.add(Dense(256, activation='relu'))
        #model.add(Dropout(0.4))
        #model.add(Dense(128, activation='relu'))
        #model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(l2_lambda)))
        model.compile(optimizer=Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model
# Training process for each category and attribute
# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        num_classes = df_attr[attr].nunique()
        binary = (num_classes == 2)
        
        # Use label encoding for all attributes
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le
        if binary:
            df_attr['label'] = df_attr['label'].astype(str)

        # Calculate class weights
       # class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(df_attr['label']), y=df_attr['label'])
        #class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_accuracy_across_folds = 0
        best_model_path = f'best_model_{category}_{attr}.h5'
        
        # Loop for each fold
        best_fold_accuracy = 0
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Create data generators
            train_generator = create_data_generator(train=True).flow_from_dataframe(
                dataframe=train_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw')

            val_generator = create_data_generator(train=False).flow_from_dataframe(
                dataframe=val_fold, directory=train_image_dir, x_col='filename', y_col='label',
                target_size=(224, 224), batch_size=32, class_mode='binary' if binary else 'raw', shuffle=False)

            # Build the model
            model = build_model(num_classes=num_classes)

            # Set up variables to track the best model in this fold
            
            best_fold_model_path = f'best_model_{category}_{attr}_fold{fold+1}.h5'
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1)
            

            # Train for multiple epochs and save the model for the best epoch in this fold
            for epoch in range(10):
                print(f"Epoch {epoch+1}/{10}")
                history = model.fit(
                    train_generator, validation_data=val_generator, epochs=1, verbose=1,callbacks=[early_stopping, reduce_lr])

                # Get the validation accuracy of the current epoch
                val_accuracy = history.history['val_accuracy'][0]

                # If the current epoch's accuracy is the best, save the model for this epoch
                if val_accuracy > best_fold_accuracy:
                    best_fold_accuracy = val_accuracy
                    model.save(best_fold_model_path)
                    print(f"New best model saved for fold {fold+1} at epoch {epoch+1} with accuracy: {val_accuracy}")

            # After the fold, check if this fold's best model is better than previous folds
            if best_fold_accuracy > best_accuracy_across_folds:
                best_accuracy_across_folds = best_fold_accuracy
                best_model_path = best_fold_model_path
                print(f"Best model across all folds so far for {category} - {attr} saved with accuracy: {best_accuracy_across_folds}")

        print(f"Overall best model for {category} - {attr} saved at: {best_model_path} with accuracy: {best_accuracy_across_folds}")
# ---- Prediction on Test Set ----

Sarees attr_10 using Resnet_50 for extracting features and predictions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    'Sarees': ['attr_10'],
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator():
    return ImageDataGenerator(rescale=1./255)

# Define feature extraction function using ResNet50
def extract_features(image_files, image_dir, model, target_size=(224, 224), batch_size=32):
    data_gen = create_data_generator()
    generator = data_gen.flow_from_dataframe(
        dataframe=image_files, directory=image_dir, x_col='filename', y_col=None,
        target_size=target_size, batch_size=batch_size, class_mode=None, shuffle=False
    )
    features = model.predict(generator, verbose=1)
    return features

# Load pre-trained ResNet50 without the fully connected layers, for feature extraction
feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        num_classes = df_attr[attr].nunique()
        binary = (num_classes == 2)
        
        # Use label encoding for all attributes
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_accuracy_across_folds = 0
        best_model_path = f'best_knn_model_{category}_{attr}.pkl'
        
        # Loop for each fold
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Extract features for train and validation sets
            train_features = extract_features(train_fold[['filename']], train_image_dir, feature_extractor)
            val_features = extract_features(val_fold[['filename']], train_image_dir, feature_extractor)
            
            # Train KNN model
            knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors based on experiments
            knn.fit(train_features, train_fold['label'])
            
            # Validate the model
            val_preds = knn.predict(val_features)
            val_accuracy = accuracy_score(val_fold['label'], val_preds)
            print(f"Validation accuracy for fold {fold+1}: {val_accuracy}")

            # Check if this fold's accuracy is the best so far
            if val_accuracy > best_accuracy_across_folds:
                best_accuracy_across_folds = val_accuracy
                # Save the model here if you want, e.g., using joblib or pickle
                # joblib.dump(knn, best_model_path)
                print(f"New best model for {category} - {attr} with accuracy: {best_accuracy_across_folds}")

        print(f"Best model across all folds for {category} - {attr} with accuracy: {best_accuracy_across_folds}")

# ---- Prediction on Test Set ----
# Example prediction flow
df_test['filename'] = df_test['id'].astype(str).apply(lambda x: x.zfill(6) + '.jpg')
for category, attributes in categories_attributes.items():
    for attr in attributes:
        print(f"Predicting on test set for category: {category}, attribute: {attr}")
        
        # Load the best model (if saved)
        # knn = joblib.load(best_model_path)
        
        # Extract features for the test set
        test_features = extract_features(df_test[['filename']], test_image_dir, feature_extractor)
        
        # Make predictions
        test_preds = knn.predict(test_features)
        
        # Convert labels back to original categories
        test_preds_decoded = label_encoders[category][attr].inverse_transform(test_preds)
        df_test[f'predicted_{attr}'] = test_preds_decoded

# Save test predictions
df_test[['id'] + [f'predicted_{attr}' for attr in categories_attributes['Sarees']]].to_csv('test_predictions.csv', index=False)
print("Test predictions saved to 'test_predictions.csv'")


Resnet_50 for extracting features and Catboost classifier for prediction

attr_5

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from catboost import CatBoostClassifier
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    'Sarees': ['attr_5'],
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator():
    return ImageDataGenerator(rescale=1./255)

# Define feature extraction function using ResNet50
def extract_features(image_files, image_dir, model, target_size=(224, 224), batch_size=32):
    data_gen = create_data_generator()
    generator = data_gen.flow_from_dataframe(
        dataframe=image_files, directory=image_dir, x_col='filename', y_col=None,
        target_size=target_size, batch_size=batch_size, class_mode=None, shuffle=False
    )
    features = model.predict(generator, verbose=1)
    return features

# Load pre-trained ResNet50 without the fully connected layers, for feature extraction
feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        
        # Encode labels
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_f1_across_folds = 0
        best_model_path = f'best_catboost_model_{category}_{attr}.cbm'
        
        # Loop for each fold
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Extract features for train and validation sets
            train_features = extract_features(train_fold[['filename']], train_image_dir, feature_extractor)
            val_features = extract_features(val_fold[['filename']], train_image_dir, feature_extractor)
            
            # Train CatBoost model with GPU support
            catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, task_type="GPU", random_seed=42, verbose=100)
            catboost_model.fit(train_features, train_fold['label'])
            
            # Validate the model with adjusted threshold
            val_probs = catboost_model.predict_proba(val_features)
            
            best_f1 = 0
            best_threshold = 0.5
            for threshold in np.arange(0.1, 0.9, 0.1):
                # Apply threshold to probabilities to get class predictions
                val_preds = (val_probs >= threshold).argmax(axis=1)
                val_f1 = f1_score(val_fold['label'], val_preds, average='weighted')
                
                if val_f1 > best_f1:
                    best_f1 = val_f1
                    best_threshold = threshold
            
            print(f"Best F1-score for fold {fold+1} with threshold {best_threshold}: {best_f1}")

            # Check if this fold's F1-score is the best so far
            if best_f1 > best_f1_across_folds:
                best_f1_across_folds = best_f1
                # Save the model with the best F1-score across folds
                catboost_model.save_model(best_model_path)
                print(f"New best model for {category} - {attr} with F1-score: {best_f1_across_folds}")

        print(f"Best model across all folds for {category} - {attr} with F1-score: {best_f1_across_folds}")

# ---- Prediction on Test Set ----
all_predictions = []

for category, attributes in categories_attributes.items():
    df_test_category = df_test[df_test['Category'] == category]
    df_test_category['id'] = df_test_category['id'].astype(str)
    df_test_category['filename'] = df_test_category['id'].apply(lambda x: x.zfill(6) + '.jpg')
    
    for attr in attributes:
        print(f"Predicting on test set for category: {category}, attribute: {attr}")
        
        # Load the best model (if saved)
        catboost_model = CatBoostClassifier()
        catboost_model.load_model(best_model_path)
        
        # Extract features for the test set
        test_features = extract_features(df_test_category[['filename']], test_image_dir, feature_extractor)
        
        # Make predictions using best threshold
        test_probs = catboost_model.predict_proba(test_features)
        test_preds = (test_probs >= best_threshold).argmax(axis=1)
        
        # Convert labels back to original categories
        test_preds_decoded = label_encoders[category][attr].inverse_transform(test_preds)
        
        # Ensure alignment between predictions and the test dataframe
        df_test_category[f'predicted_{attr}'] = test_preds_decoded
        
        # Append the predictions to the main dataframe
        all_predictions.append(df_test_category[['id', f'predicted_{attr}']])

# Concatenate all the predictions and save to a CSV
df_predictions = pd.concat(all_predictions)
df_predictions.to_csv('Sarees_f1_cat_attr_5.csv', index=False)

print("Test predictions saved to 'Sarees_f1_cat_attr_5.csv'")


attr_6

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from catboost import CatBoostClassifier
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    'Sarees': ['attr_6'],
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator():
    return ImageDataGenerator(rescale=1./255)

# Define feature extraction function using ResNet50
def extract_features(image_files, image_dir, model, target_size=(224, 224), batch_size=32):
    data_gen = create_data_generator()
    generator = data_gen.flow_from_dataframe(
        dataframe=image_files, directory=image_dir, x_col='filename', y_col=None,
        target_size=target_size, batch_size=batch_size, class_mode=None, shuffle=False
    )
    features = model.predict(generator, verbose=1)
    return features

# Load pre-trained ResNet50 without the fully connected layers, for feature extraction
feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        
        # Encode labels
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_f1_across_folds = 0
        best_model_path = f'best_catboost_model_{category}_{attr}.cbm'
        
        # Loop for each fold
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Extract features for train and validation sets
            train_features = extract_features(train_fold[['filename']], train_image_dir, feature_extractor)
            val_features = extract_features(val_fold[['filename']], train_image_dir, feature_extractor)
            
            # Train CatBoost model with GPU support
            catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, task_type="GPU", random_seed=42, verbose=100)
            catboost_model.fit(train_features, train_fold['label'])
            
            # Validate the model with adjusted threshold
            val_probs = catboost_model.predict_proba(val_features)
            
            best_f1 = 0
            best_threshold = 0.5
            for threshold in np.arange(0.1, 0.9, 0.1):
                # Apply threshold to probabilities to get class predictions
                val_preds = (val_probs >= threshold).argmax(axis=1)
                val_f1 = f1_score(val_fold['label'], val_preds, average='weighted')
                
                if val_f1 > best_f1:
                    best_f1 = val_f1
                    best_threshold = threshold
            
            print(f"Best F1-score for fold {fold+1} with threshold {best_threshold}: {best_f1}")

            # Check if this fold's F1-score is the best so far
            if best_f1 > best_f1_across_folds:
                best_f1_across_folds = best_f1
                # Save the model with the best F1-score across folds
                catboost_model.save_model(best_model_path)
                print(f"New best model for {category} - {attr} with F1-score: {best_f1_across_folds}")

        print(f"Best model across all folds for {category} - {attr} with F1-score: {best_f1_across_folds}")

# ---- Prediction on Test Set ----
all_predictions = []

for category, attributes in categories_attributes.items():
    df_test_category = df_test[df_test['Category'] == category]
    df_test_category['id'] = df_test_category['id'].astype(str)
    df_test_category['filename'] = df_test_category['id'].apply(lambda x: x.zfill(6) + '.jpg')
    
    for attr in attributes:
        print(f"Predicting on test set for category: {category}, attribute: {attr}")
        
        # Load the best model (if saved)
        catboost_model = CatBoostClassifier()
        catboost_model.load_model(best_model_path)
        
        # Extract features for the test set
        test_features = extract_features(df_test_category[['filename']], test_image_dir, feature_extractor)
        
        # Make predictions using best threshold
        test_probs = catboost_model.predict_proba(test_features)
        test_preds = (test_probs >= best_threshold).argmax(axis=1)
        
        # Convert labels back to original categories
        test_preds_decoded = label_encoders[category][attr].inverse_transform(test_preds)
        
        # Ensure alignment between predictions and the test dataframe
        df_test_category[f'predicted_{attr}'] = test_preds_decoded
        
        # Append the predictions to the main dataframe
        all_predictions.append(df_test_category[['id', f'predicted_{attr}']])

# Concatenate all the predictions and save to a CSV
df_predictions = pd.concat(all_predictions)
df_predictions.to_csv('Sarees_f1_cat_attr_6.csv', index=False)

print("Test predictions saved to 'Sarees_f1_cat_attr_6.csv'")


attr_7

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from catboost import CatBoostClassifier
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    'Sarees': ['attr_7'],
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator():
    return ImageDataGenerator(rescale=1./255)

# Define feature extraction function using ResNet50
def extract_features(image_files, image_dir, model, target_size=(224, 224), batch_size=32):
    data_gen = create_data_generator()
    generator = data_gen.flow_from_dataframe(
        dataframe=image_files, directory=image_dir, x_col='filename', y_col=None,
        target_size=target_size, batch_size=batch_size, class_mode=None, shuffle=False
    )
    features = model.predict(generator, verbose=1)
    return features

# Load pre-trained ResNet50 without the fully connected layers, for feature extraction
feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        
        # Encode labels
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_f1_across_folds = 0
        best_model_path = f'best_catboost_model_{category}_{attr}.cbm'
        
        # Loop for each fold
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Extract features for train and validation sets
            train_features = extract_features(train_fold[['filename']], train_image_dir, feature_extractor)
            val_features = extract_features(val_fold[['filename']], train_image_dir, feature_extractor)
            
            # Train CatBoost model with GPU support
            catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, task_type="GPU", random_seed=42, verbose=100)
            catboost_model.fit(train_features, train_fold['label'])
            
            # Validate the model with adjusted threshold
            val_probs = catboost_model.predict_proba(val_features)
            
            best_f1 = 0
            best_threshold = 0.5
            for threshold in np.arange(0.1, 0.9, 0.1):
                # Apply threshold to probabilities to get class predictions
                val_preds = (val_probs >= threshold).argmax(axis=1)
                val_f1 = f1_score(val_fold['label'], val_preds, average='weighted')
                
                if val_f1 > best_f1:
                    best_f1 = val_f1
                    best_threshold = threshold
            
            print(f"Best F1-score for fold {fold+1} with threshold {best_threshold}: {best_f1}")

            # Check if this fold's F1-score is the best so far
            if best_f1 > best_f1_across_folds:
                best_f1_across_folds = best_f1
                # Save the model with the best F1-score across folds
                catboost_model.save_model(best_model_path)
                print(f"New best model for {category} - {attr} with F1-score: {best_f1_across_folds}")

        print(f"Best model across all folds for {category} - {attr} with F1-score: {best_f1_across_folds}")

# ---- Prediction on Test Set ----
all_predictions = []

for category, attributes in categories_attributes.items():
    df_test_category = df_test[df_test['Category'] == category]
    df_test_category['id'] = df_test_category['id'].astype(str)
    df_test_category['filename'] = df_test_category['id'].apply(lambda x: x.zfill(6) + '.jpg')
    
    for attr in attributes:
        print(f"Predicting on test set for category: {category}, attribute: {attr}")
        
        # Load the best model (if saved)
        catboost_model = CatBoostClassifier()
        catboost_model.load_model(best_model_path)
        
        # Extract features for the test set
        test_features = extract_features(df_test_category[['filename']], test_image_dir, feature_extractor)
        
        # Make predictions using best threshold
        test_probs = catboost_model.predict_proba(test_features)
        test_preds = (test_probs >= best_threshold).argmax(axis=1)
        
        # Convert labels back to original categories
        test_preds_decoded = label_encoders[category][attr].inverse_transform(test_preds)
        
        # Ensure alignment between predictions and the test dataframe
        df_test_category[f'predicted_{attr}'] = test_preds_decoded
        
        # Append the predictions to the main dataframe
        all_predictions.append(df_test_category[['id', f'predicted_{attr}']])

# Concatenate all the predictions and save to a CSV
df_predictions = pd.concat(all_predictions)
df_predictions.to_csv('Sarees_f1_cat_attr_7.csv', index=False)

print("Test predictions saved to 'Sarees_f1_cat_attr_7.csv'")


attr_8

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from catboost import CatBoostClassifier
import os

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    'Sarees': ['attr_8'],
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator():
    return ImageDataGenerator(rescale=1./255)

# Define feature extraction function using ResNet50
def extract_features(image_files, image_dir, model, target_size=(224, 224), batch_size=32):
    data_gen = create_data_generator()
    generator = data_gen.flow_from_dataframe(
        dataframe=image_files, directory=image_dir, x_col='filename', y_col=None,
        target_size=target_size, batch_size=batch_size, class_mode=None, shuffle=False
    )
    features = model.predict(generator, verbose=1)
    return features

# Load pre-trained ResNet50 without the fully connected layers, for feature extraction
feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        
        # Encode labels
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_f1_across_folds = 0
        best_model_path = f'best_catboost_model_{category}_{attr}.cbm'
        
        # Loop for each fold
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Extract features for train and validation sets
            train_features = extract_features(train_fold[['filename']], train_image_dir, feature_extractor)
            val_features = extract_features(val_fold[['filename']], train_image_dir, feature_extractor)
            
            # Train CatBoost model with GPU support
            catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, task_type="GPU", random_seed=42, verbose=100)
            catboost_model.fit(train_features, train_fold['label'])
            
            # Validate the model with adjusted threshold
            val_probs = catboost_model.predict_proba(val_features)
            
            best_f1 = 0
            best_threshold = 0.5
            for threshold in np.arange(0.1, 0.9, 0.1):
                # Apply threshold to probabilities to get class predictions
                val_preds = (val_probs >= threshold).argmax(axis=1)
                val_f1 = f1_score(val_fold['label'], val_preds, average='weighted')
                
                if val_f1 > best_f1:
                    best_f1 = val_f1
                    best_threshold = threshold
            
            print(f"Best F1-score for fold {fold+1} with threshold {best_threshold}: {best_f1}")

            # Check if this fold's F1-score is the best so far
            if best_f1 > best_f1_across_folds:
                best_f1_across_folds = best_f1
                # Save the model with the best F1-score across folds
                catboost_model.save_model(best_model_path)
                print(f"New best model for {category} - {attr} with F1-score: {best_f1_across_folds}")

        print(f"Best model across all folds for {category} - {attr} with F1-score: {best_f1_across_folds}")

# ---- Prediction on Test Set ----
all_predictions = []

for category, attributes in categories_attributes.items():
    df_test_category = df_test[df_test['Category'] == category]
    df_test_category['id'] = df_test_category['id'].astype(str)
    df_test_category['filename'] = df_test_category['id'].apply(lambda x: x.zfill(6) + '.jpg')
    
    for attr in attributes:
        print(f"Predicting on test set for category: {category}, attribute: {attr}")
        
        # Load the best model (if saved)
        catboost_model = CatBoostClassifier()
        catboost_model.load_model(best_model_path)
        
        # Extract features for the test set
        test_features = extract_features(df_test_category[['filename']], test_image_dir, feature_extractor)
        
        # Make predictions using best threshold
        test_probs = catboost_model.predict_proba(test_features)
        test_preds = (test_probs >= best_threshold).argmax(axis=1)
        
        # Convert labels back to original categories
        test_preds_decoded = label_encoders[category][attr].inverse_transform(test_preds)
        
        # Ensure alignment between predictions and the test dataframe
        df_test_category[f'predicted_{attr}'] = test_preds_decoded
        
        # Append the predictions to the main dataframe
        all_predictions.append(df_test_category[['id', f'predicted_{attr}']])

# Concatenate all the predictions and save to a CSV
df_predictions = pd.concat(all_predictions)
df_predictions.to_csv('Sarees_f1_cat_attr_8.csv', index=False)

print("Test predictions saved to 'Sarees_f1_cat_attr_8.csv'")


Resnet_50 for extracting features and Random Forest for predictions

attr_6

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pickle

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    'Sarees': ['attr_6'],
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator():
    return ImageDataGenerator(rescale=1./255)

# Define feature extraction function using ResNet50
def extract_features(image_files, image_dir, model, target_size=(224, 224), batch_size=32):
    data_gen = create_data_generator()
    generator = data_gen.flow_from_dataframe(
        dataframe=image_files, directory=image_dir, x_col='filename', y_col=None,
        target_size=target_size, batch_size=batch_size, class_mode=None, shuffle=False
    )
    features = model.predict(generator, verbose=1)
    return features

# Load pre-trained ResNet50 without the fully connected layers, for feature extraction
feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        
        # Encode labels
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_f1_across_folds = 0
        best_model_path = f'best_rf_model_{category}_{attr}.pkl'
        
        # Loop for each fold
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Extract features for train and validation sets
            train_features = extract_features(train_fold[['filename']], train_image_dir, feature_extractor)
            val_features = extract_features(val_fold[['filename']], train_image_dir, feature_extractor)
            
            # Train RandomForest model
            rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
            rf_model.fit(train_features, train_fold['label'])
            
            # Validate the model with adjusted threshold
            val_probs = rf_model.predict_proba(val_features)
            
            best_f1 = 0
            best_threshold = 0.5
            for threshold in np.arange(0.1, 0.9, 0.1):
                # Apply threshold to probabilities to get class predictions
                val_preds = (val_probs >= threshold).argmax(axis=1)
                val_f1 = f1_score(val_fold['label'], val_preds, average='weighted')
                
                if val_f1 > best_f1:
                    best_f1 = val_f1
                    best_threshold = threshold
            
            print(f"Best F1-score for fold {fold+1} with threshold {best_threshold}: {best_f1}")

            # Check if this fold's F1-score is the best so far
            if best_f1 > best_f1_across_folds:
                best_f1_across_folds = best_f1
                # Save the model with the best F1-score across folds
                with open(best_model_path, 'wb') as f:
                    pickle.dump(rf_model, f)
                print(f"New best model for {category} - {attr} with F1-score: {best_f1_across_folds}")

        print(f"Best model across all folds for {category} - {attr} with F1-score: {best_f1_across_folds}")

# ---- Prediction on Test Set ----
all_predictions = []

for category, attributes in categories_attributes.items():
    df_test_category = df_test[df_test['Category'] == category]
    df_test_category['id'] = df_test_category['id'].astype(str)
    df_test_category['filename'] = df_test_category['id'].apply(lambda x: x.zfill(6) + '.jpg')
    
    for attr in attributes:
        print(f"Predicting on test set for category: {category}, attribute: {attr}")
        
        # Load the best model (if saved)
        with open(best_model_path, 'rb') as f:
            rf_model = pickle.load(f)
        
        # Extract features for the test set
        test_features = extract_features(df_test_category[['filename']], test_image_dir, feature_extractor)
        
        # Make predictions using best threshold
        test_probs = rf_model.predict_proba(test_features)
        test_preds = (test_probs >= best_threshold).argmax(axis=1)
        
        # Convert labels back to original categories
        test_preds_decoded = label_encoders[category][attr].inverse_transform(test_preds)
        
        # Ensure alignment between predictions and the test dataframe
        df_test_category[f'predicted_{attr}'] = test_preds_decoded
        
        # Append the predictions to the main dataframe
        all_predictions.append(df_test_category[['id', f'predicted_{attr}']])

# Concatenate all the predictions and save to a CSV
df_predictions = pd.concat(all_predictions)
df_predictions.to_csv('Sarees_f1_rf_attr_6.csv', index=False)

print("Test predictions saved to 'Sarees_f1_rf_attr_6.csv'")


attr_7

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pickle

# Load train and test sets
df_train = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/meesho/visual-taxonomy/test.csv')
train_image_dir = '/kaggle/input/meesho/visual-taxonomy/train_images'
test_image_dir = '/kaggle/input/meesho/visual-taxonomy/test_images'

# Define attribute configurations for each category
categories_attributes = {
    'Sarees': ['attr_7'],
}

# Dictionary to store encoders for each category and attribute
label_encoders = {}

# Create a data generator
def create_data_generator():
    return ImageDataGenerator(rescale=1./255)

# Define feature extraction function using ResNet50
def extract_features(image_files, image_dir, model, target_size=(224, 224), batch_size=32):
    data_gen = create_data_generator()
    generator = data_gen.flow_from_dataframe(
        dataframe=image_files, directory=image_dir, x_col='filename', y_col=None,
        target_size=target_size, batch_size=batch_size, class_mode=None, shuffle=False
    )
    features = model.predict(generator, verbose=1)
    return features

# Load pre-trained ResNet50 without the fully connected layers, for feature extraction
feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Training process for each category and attribute
for category, attributes in categories_attributes.items():
    df_category = df_train[df_train['Category'] == category]
    df_category['id'] = df_category['id'].astype(str)
    df_category['filename'] = df_category['id'].apply(lambda x: x.zfill(6) + '.jpg')

    label_encoders[category] = {}

    # Iterate over each attribute
    for attr in attributes:
        print(f"Training for category: {category}, attribute: {attr}")
        
        # Drop columns not relevant to the current attribute and handle NaNs
        df_attr = df_category[['filename', attr]].dropna()
        
        # Encode labels
        le = LabelEncoder()
        df_attr['label'] = le.fit_transform(df_attr[attr])
        label_encoders[category][attr] = le

        # Set up K-Fold Cross-Validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        best_f1_across_folds = 0
        best_model_path = f'best_rf_model_{category}_{attr}.pkl'
        
        # Loop for each fold
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_attr)):
            print(f"Training fold {fold+1} for {category} - {attr}")
            train_fold = df_attr.iloc[train_idx]
            val_fold = df_attr.iloc[val_idx]

            # Extract features for train and validation sets
            train_features = extract_features(train_fold[['filename']], train_image_dir, feature_extractor)
            val_features = extract_features(val_fold[['filename']], train_image_dir, feature_extractor)
            
            # Train RandomForest model
            rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
            rf_model.fit(train_features, train_fold['label'])
            
            # Validate the model with adjusted threshold
            val_probs = rf_model.predict_proba(val_features)
            
            best_f1 = 0
            best_threshold = 0.5
            for threshold in np.arange(0.1, 0.9, 0.1):
                # Apply threshold to probabilities to get class predictions
                val_preds = (val_probs >= threshold).argmax(axis=1)
                val_f1 = f1_score(val_fold['label'], val_preds, average='weighted')
                
                if val_f1 > best_f1:
                    best_f1 = val_f1
                    best_threshold = threshold
            
            print(f"Best F1-score for fold {fold+1} with threshold {best_threshold}: {best_f1}")

            # Check if this fold's F1-score is the best so far
            if best_f1 > best_f1_across_folds:
                best_f1_across_folds = best_f1
                # Save the model with the best F1-score across folds
                with open(best_model_path, 'wb') as f:
                    pickle.dump(rf_model, f)
                print(f"New best model for {category} - {attr} with F1-score: {best_f1_across_folds}")

        print(f"Best model across all folds for {category} - {attr} with F1-score: {best_f1_across_folds}")

# ---- Prediction on Test Set ----
all_predictions = []

for category, attributes in categories_attributes.items():
    df_test_category = df_test[df_test['Category'] == category]
    df_test_category['id'] = df_test_category['id'].astype(str)
    df_test_category['filename'] = df_test_category['id'].apply(lambda x: x.zfill(6) + '.jpg')
    
    for attr in attributes:
        print(f"Predicting on test set for category: {category}, attribute: {attr}")
        
        # Load the best model (if saved)
        with open(best_model_path, 'rb') as f:
            rf_model = pickle.load(f)
        
        # Extract features for the test set
        test_features = extract_features(df_test_category[['filename']], test_image_dir, feature_extractor)
        
        # Make predictions using best threshold
        test_probs = rf_model.predict_proba(test_features)
        test_preds = (test_probs >= best_threshold).argmax(axis=1)
        
        # Convert labels back to original categories
        test_preds_decoded = label_encoders[category][attr].inverse_transform(test_preds)
        
        # Ensure alignment between predictions and the test dataframe
        df_test_category[f'predicted_{attr}'] = test_preds_decoded
        
        # Append the predictions to the main dataframe
        all_predictions.append(df_test_category[['id', f'predicted_{attr}']])

# Concatenate all the predictions and save to a CSV
df_predictions = pd.concat(all_predictions)
df_predictions.to_csv('Sarees_f1_rf_attr_7.csv', index=False)

print("Test predictions saved to 'Sarees_f1_rf_attr_7.csv'")
